<a href="https://colab.research.google.com/github/Yorrek/SchlemmerSlammer/blob/master/epicurious_tsne002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MAT 259: Final Project

Chantal Nguyen

Instructor: George Legrady


The following code reads in a json file containing recipes extracted from the online recipe database Epicurious.com. The data includes recipe titles, ratings, review counts, dates of publication, ingredient lists, instructions, etc. 

tf-idf is used to vectorize the ingredient lists of each recipe. Dimensionality reduction is performed first with SVD and then with t-SNE to project recipes into 2D space. k-means clustering is used to cluster recipes into 15 clusters. 

In [0]:
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import io
import re
import json
import pandas as pd
import time
from datetime import datetime

Load in the json file as a pandas dataframe

In [0]:
# load in the data
data = pd.read_json('epicurious-recipes.json',lines=True)
data = data[data.reviewsCount > 0]
for index, row in data.iterrows(): # clean up entries with no ingredients listed
    if not isinstance(row['ingredients'],(list,)):
        data = data.drop(index)
data = data.reset_index(drop=True)

Extract ingredient lists and remove numbers and non-alphabetic characters

In [0]:
ingredients_data = []
chars_to_remove = dict((ord(char), None) for char in '/0123456789!"#%\'()*+,-./:;<=>?@[\]^_`{|}~')
for index, row in data.iterrows():
    ingredients_data.append((''.join(row['ingredients'])).translate(chars_to_remove))

Define stopwords: common English stopwords (found here: https://github.com/stopwords-iso/stopwords-en/) plus words related to measurement quantities

In [0]:
with open('stopwords-en.json') as f:
    stopwords = json.load(f)

In [0]:
stopwords = stopwords + [u'cup', u'cups', u'tablespoon', u'tablespoons', u'teaspoon', u'teaspoons', u'c', u'tbsp', u'tsp', u'oz', u'g', u'kg', u'lb', u'pt', u'gal', u'qt', u'qts', u'tbsps', u'tsps', u'ml', u'l', u'inch', u'inches', u'pinch', u'pinches', u'dash', u'dashes', u'ounce', u'ounces', u'can', u'cans', u'bag', u'bags', u'package', u'packages', u'gram', u'grams', u'pound', u'pounds']
stopwords = stopwords + [u'purpose', u'allpurpose', u'salted', u'unsalted', u'extract', u'stick', u'temperature', u'powder',u'peeled']

Vectorize with tf-idf

In [9]:
vectorizer = TfidfVectorizer(
    max_df=0.5, 
    max_features=10000,
    min_df=2, 
    stop_words=stopwords,
    use_idf=True
    )
X = vectorizer.fit_transform(ingredients_data)

/usr/local/lib/python2.7/dist-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens [u'ain', u'daren', u'hadn', u'herse', u'himse', u'itse', u'mayn', u'mightn', u'mon', u'mustn', u'myse', u'needn', u'oughtn', u'shan'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


First reduce dimensionality to 50 dimensions with SVD (performing t-SNE now would be too intensive)

In [0]:
svd = TruncatedSVD(n_components=50)
Y = svd.fit_transform(X)

Reduce to 2D with t-SNE

In [11]:
tsne_model = TSNE(n_components=2, verbose=1, random_state=1)
Z = tsne_model.fit_transform(Y)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 31417 samples in 0.083s...
[t-SNE] Computed neighbors for 31417 samples in 117.918s...
[t-SNE] Computed conditional probabilities for sample 1000 / 31417
[t-SNE] Computed conditional probabilities for sample 2000 / 31417
[t-SNE] Computed conditional probabilities for sample 3000 / 31417
[t-SNE] Computed conditional probabilities for sample 4000 / 31417
[t-SNE] Computed conditional probabilities for sample 5000 / 31417
[t-SNE] Computed conditional probabilities for sample 6000 / 31417
[t-SNE] Computed conditional probabilities for sample 7000 / 31417
[t-SNE] Computed conditional probabilities for sample 8000 / 31417
[t-SNE] Computed conditional probabilities for sample 9000 / 31417
[t-SNE] Computed conditional probabilities for sample 10000 / 31417
[t-SNE] Computed conditional probabilities for sample 11000 / 31417
[t-SNE] Computed conditional probabilities for sample 12000 / 31417
[t-SNE] Computed conditional probabilities for s

Extract t-SNE coordinates, append to dataframe

In [0]:
x_coords = Z[:,0]
y_coords = Z[:,1]
coords = np.transpose(np.vstack((x_coords, y_coords)))  
with open('epi_coords.csv', 'w') as f:
    for x, y in coords:
        f.write('%f, %f\n' % (x,y)) 

In [0]:
coords_df = pd.DataFrame(np.stack((x_coords,y_coords)).transpose(), columns=list('xy'))

In [0]:
data = data[['hed', 'pubDate', 'aggregateRating','reviewsCount','willMakeAgainPct']]
data = data.join(coords_df)

Perform k-means clustering into (arbitrarily-set) 15 clusters

In [20]:
num_clusters = 12
km = KMeans(
      n_clusters=num_clusters, 
      init='k-means++', 
      max_iter=100, 
      n_init=1)
print("Clustering sparse data with %s" % km)
km.fit(X)

print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(num_clusters):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])
        

Clustering sparse data with KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=12, n_init=1, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
Top terms per cluster:
Cluster 0:
 mustard
 dijon
 vinegar
 chopped
 oil
 olive
 pepper
 minced
 lemon
 black
Cluster 1:
 chicken
 broth
 chopped
 lowsalt
 canned
 white
 dried
 oil
 butter
 garlic
Cluster 2:
 olive
 oil
 garlic
 chopped
 extravirgin
 minced
 leaves
 pepper
 cloves
 dried
Cluster 3:
 chocolate
 unsweetened
 bittersweet
 semisweet
 sugar
 cream
 vanilla
 chopped
 butter
 cocoa
Cluster 4:
 chopped
 finely
 onion
 pepper
 oil
 ground
 garlic
 red
 olive
 diced
Cluster 5:
 ground
 salt
 water
 butter
 pepper
 black
 sugar
 oil
 chopped
 flour
Cluster 6:
 sugar
 cream
 vanilla
 egg
 heavy
 milk
 butter
 yolks
 whipping
 chilled
Cluster 7:
 baking
 flour
 sugar
 butter
 salt
 soda
 vanilla
 eggs
 brown
 ground
Cluster 8:
 sliced
 thinly
 red
 oil
 chopped
 olive
 

In [0]:
data['label'] = km.labels_

Do some cleanup and save data as json file containing recipe title, rating, review count, publication date, % of reviewers who said they will make the recipe again, t-SNE coords, and cluster label

In [27]:
datestrings = []
for index, row in data.iterrows():
    pdate = row['pubDate']
    data.at[index,'pubDate'] = time.mktime(datetime.strptime(pdate, '%Y-%m-%dT%H:%M:%S.%fZ').timetuple())/1e+09
    datestrings.append(pd.to_datetime.pdate[0:10])
data['datestr'] = datestrings

TypeError: ignored

In [0]:
with open('epicurious_data.json','w') as f:
    f.write(data.to_json(orient = "records"))

Dieses Datenset (epicurious_data.json) muss nun heruntergeladen werden und kann mit Processing visualisiert werden